# Preprocessing

In [53]:
import heapq
import os
import sys
import numpy as np
import pandas as pd
import tifffile as tif
import h5py
from skimage import io, img_as_float32
import matplotlib.pyplot as plt
from scipy import ndimage
import IPython.display
# import PIL
%matplotlib inline
%load_ext autoreload
%autoreload 2
import os, sys
sys.path.append("../bardensr")
import bardensr
import bardensr.plotting

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [5]:
n_channels = 2
n_cycles = 5

img = tif.imread('./data/crisprmap20210506/amp-ch1-cycle0.tiff')
img_size = img.shape[:2]

X = np.zeros((n_channels*n_cycles, 1, *img_size))
k = 0
for i in range(n_channels):
    for j in range(n_cycles):
        fn = f'./data/crisprmap20210506/amp-ch{i+1}-cycle{j}.tiff'
        print(fn)
        img = tif.imread(fn)
        X[k, 0, :, :] = img[:, :, 1]  # using layer 1 out of layers 0-3
        k = k+1

./data/crisprmap20210506/amp-ch1-cycle0.tiff
./data/crisprmap20210506/amp-ch1-cycle1.tiff
./data/crisprmap20210506/amp-ch1-cycle2.tiff
./data/crisprmap20210506/amp-ch1-cycle3.tiff
./data/crisprmap20210506/amp-ch1-cycle4.tiff
./data/crisprmap20210506/amp-ch2-cycle0.tiff
./data/crisprmap20210506/amp-ch2-cycle1.tiff
./data/crisprmap20210506/amp-ch2-cycle2.tiff
./data/crisprmap20210506/amp-ch2-cycle3.tiff
./data/crisprmap20210506/amp-ch2-cycle4.tiff


In [6]:
img.shape

(2048, 1792, 4)

In [7]:
X.shape

(10, 1, 2048, 1792)

In [8]:
# for i in range(X.shape[0]):
#     plt.figure(dpi=250)
#     plt.imshow(X[i, 0,])

## Check how many pixels are roughly in one amplicon

In [9]:
# img = tif.imread('./data/crisprmap20210506/amp-ch1-cycle2.tiff')
# plt.figure(dpi=100)
# plt.imshow(img[80:100, 80:100, 1])
# plt.axis('image')
# # im.view()

This shows if the windows for local maxima is 3 by 3 pixels, they should be right for catching one amplicon

## Remove border artifact and get X only within the range of the slide - segmentation images must be set to the same range when overlaying

In [10]:
up = 10
down = 1900
left = 50
right = 1650

Xcenter = X[:, :, up:down, left:right] # range is selected based on observation of images

In [11]:
# for i in range(Xcenter.shape[0]):
#     plt.figure(dpi=250)
#     plt.imshow(Xcenter[i, 0,])

# Codebook

The most important is to make sure the codebook and the images are of the same order

In [12]:
# !! currently the first value shows nan and hard code it to be 1; needs a better fix
codebook = np.genfromtxt('./data/CRISPRmap_pilot_codebook_default.csv', filling_values=1, dtype=np.int, delimiter=",")
codebook

array([[1, 0, 0, 0, 0, 1, 0, 0, 0, 0],
       [0, 0, 0, 1, 0, 0, 0, 0, 1, 0],
       [0, 1, 0, 0, 0, 0, 1, 0, 0, 0],
       [0, 0, 0, 0, 1, 0, 0, 0, 0, 1],
       [0, 0, 1, 0, 0, 0, 0, 1, 0, 0],
       [0, 1, 0, 0, 0, 1, 0, 0, 0, 0],
       [0, 0, 1, 0, 0, 1, 0, 0, 0, 0],
       [1, 0, 0, 0, 0, 0, 1, 0, 0, 0],
       [0, 0, 1, 0, 0, 0, 1, 0, 0, 0],
       [0, 1, 0, 0, 0, 0, 0, 1, 0, 0]])

In [13]:
# codebook = csv.reshape((10, 2, 5)).T.astype(bool)
# codebook

In [14]:
codebook.shape

(10, 10)

The order of the codebook matches the order of the image stack.

## Background subtraction and normalization

In [15]:
Xcenter.shape

(10, 1, 1890, 1600)

In [16]:
# Different from how bardensr did it (use these functions for now because they are tested)
Xnorm = bardensr.preprocessing.background_subtraction(Xcenter, [0,10,10])
Xnorm = bardensr.preprocessing.minmax(Xnorm)

In [17]:
Xnorm.shape

(10, 1, 1890, 1600)

In [18]:
# for i in range(Xnorm.shape[0]):
#     plt.figure(dpi=250)
#     plt.imshow(Xnorm[i, 0,])

## Distribution

In [19]:
# # create the histogram
# for i in range(Xnorm.shape[0]):
#     histogram, bin_edges = np.histogram(Xnorm[i, 0], bins=256, range=(0, 1))

#     # configure and draw the histogram figure
#     plt.figure()
# #     plt.title("Grayscale Histogram")
#     plt.xlabel("grayscale value")
#     plt.ylabel("pixels")

#     plt.semilogy(bin_edges[0:-1], histogram)  # <- or here

#     plt.xlim([0.0, 1.0])  # <- named arguments do not work here
#     plt.show()

## Thresholding

In [20]:
Xnorm.shape

(10, 1, 1890, 1600)

In [21]:
Xnorm[0, 0,].shape

(1890, 1600)

In [22]:
# upper = 0.9
# lower = [0.2, 0.16, 0.13, 0.15, 0.16, 0.16, 0.16, 0.13, 0.15, 0.15]
# Xthresh = Xnorm.copy()
# for i in range(Xthresh.shape[0]):
#     single = Xthresh[i, 0,]
#     single[single < lower[i]] = 0
#     single[single > upper] = upper
#     Xthresh[i, 0] = single

In [23]:
residual = 0.125  # tested through the spot method and the cell method should be less sensitive 
Xthresh = Xnorm.copy()
Xthresh[Xthresh < residual] = 0

In [24]:
# for i in range(Xthresh.shape[0]):
#     histogram, bin_edges = np.histogram(Xthresh[i, 0], bins=256, range=(0, 1))

#     # configure and draw the histogram figure
#     plt.figure()
# #     plt.title("Grayscale Histogram")
#     plt.xlabel("grayscale value")
#     plt.ylabel("pixels")

#     plt.semilogy(bin_edges[0:-1], histogram)  # <- or here

#     plt.xlim([0.0, 1.0])  # <- named arguments do not work here
#     plt.show()

In [25]:
# # Before and after (partial region)
# # with bardensr.plotting.AnimAcross() as a:
# for i in range(Xthresh.shape[0]):
#     plt.figure(dpi=250)
#     plt.subplot(1, 2, 1)
# #     a('before')
#     plt.imshow(Xnorm[i, 0, 200:500, 200:500])
#     plt.subplot(1, 2, 2)
# #     a('after')
#     plt.imshow(Xthresh[i, 0, 200:500, 200:500])

# Peak calling and counting (inside membrane masks)
sun, mon

**Check by overlay to see if there are enough spots called.**

In [26]:
# Read in the membrane segmentation
hf = h5py.File("./data/Ben_MembraneSegmentation_NuclearGFP_2021-06-22.hdf5", "r")
print(hf.keys())
masks_mem = hf["Cell Seg Mask"]

<KeysViewHDF5 ['Average GFP Masks', 'Cell Seg Mask', 'Max Projection', 'Nuc Seg Mask', 'Summed GFP Masks']>


In [27]:
# plt.figure(dpi=250)
# plt.imshow(masks_mem)
# plt.axis('image')

In [28]:
# Find the number of membrane segmentation masks
np.unique(masks_mem)

array([  0,   1,   2,   3,   4,   5,   6,   7,   8,   9,  10,  11,  12,
        13,  14,  15,  16,  17,  18,  19,  20,  21,  22,  23,  24,  25,
        26,  27,  28,  29,  30,  31,  32,  33,  34,  35,  36,  37,  38,
        39,  40,  41,  42,  43,  44,  45,  46,  47,  48,  49,  50,  51,
        52,  53,  54,  55,  56,  57,  58,  59,  60,  61,  62,  63,  64,
        65,  66,  67,  68,  69,  70,  71,  72,  73,  74,  75,  76,  77,
        78,  79,  80,  81,  82,  83,  84,  85,  86,  87,  88,  89,  90,
        91,  92,  93,  94,  95,  96,  97,  98,  99, 100, 101, 102, 103,
       104, 105, 106, 107, 108, 109, 110, 111, 112, 113, 114, 115, 116,
       117, 118, 119, 120, 121, 122, 123, 124, 125, 126, 127, 128, 129,
       130, 131, 132, 133, 134, 135, 136, 137, 138, 139, 140, 141, 142,
       143, 144, 145, 146, 147, 148, 149, 150, 151, 152, 153, 154, 155,
       156, 157, 158, 159, 160, 161, 162, 163, 164, 165, 166, 167, 168,
       169, 170, 171, 172, 173, 174, 175, 176, 177, 178, 179, 18

In [29]:
# # Check starting from which number the masks are truly membrane segmentation (instead of the background)
# plt.figure(dpi=300)
# for i in range(0, 5):
#     plt.figure()
#     plt.imshow(np.array(masks_mem)==i)

In [30]:
#     xindex = test.loc[i, 'm2'] + left
#     yindex = test.loc[i, 'm1'] + up

In [31]:
# pd.DataFrame(np.zeros((len(np.unique(masks_mem)), Xthresh.shape[0])))

In [32]:
# cell_table = pd.DataFrame(np.zeros((len(np.unique(masks_mem)), Xthresh.shape[0])))
cell_table = np.zeros((len(np.unique(masks_mem)), Xthresh.shape[0]))

for k in range(Xthresh.shape[0]):  # for the kth image
    # Get local maximum values of desired neighborhood (size of the amplicons)
    max_fil = ndimage.maximum_filter(Xthresh[k,], size=(1, 2, 2))

    # Threshold the image to find locations of interest
    # assuming 6 standard deviations above the mean of the image
    peak_thresh = max_fil.mean() + max_fil.std() * 6

    # find areas greater than peak_thresh
    labels, num_labels = ndimage.label(max_fil > peak_thresh)

    # Get the positions of the maxima
    coords = ndimage.measurements.center_of_mass(Xthresh[k,], 
                                                 labels=labels, 
                                                 index=np.arange(1, num_labels + 1))

    # # Get the maximum value in the labels
    # values = ndimage.measurements.maximum(img, labels=labels, index=np.arange(1, num_labels + 1))
    # # https://stackoverflow.com/questions/55453110/how-to-find-local-maxima-of-3d-array-in-python

    for _, m1, m2 in coords:
        m1 = int(np.round(m1))
        m2 = int(np.round(m2))
        mem_id = masks_mem[m1+up, m2+left]  # important to match the coordinates if images are trimmed
        if mem_id>0: # 0 is background
#             cell_table.loc[mem_id, k] += 1
            cell_table[mem_id, k] += 1

In [33]:
# labels

In [34]:
# num_labels

In [35]:
coords[:10]

[(0.0, 6.543602130081081, 861.0),
 (0.0, 8.0, 108.59272456891979),
 (0.0, 9.481436771695659, 903.0),
 (0.0, 11.254144676468409, 729.667745198682),
 (0.0, 14.440051349491345, 1046.6361900533907),
 (0.0, 14.434706722909056, 1560.5603333633226),
 (0.0, 30.0, 542.0),
 (0.0, 30.435710348342994, 854.9999999999999),
 (0.0, 34.46947260290432, 990.7079067808304),
 (0.0, 34.362508995111035, 1529.6088846841628)]

In [36]:
int(np.round(801.5228444869555))

802

In [37]:
len(coords)

299

In [38]:
cell_table

array([[ 0.,  0.,  0., ...,  0.,  0.,  0.],
       [ 0.,  0.,  0., ...,  0.,  0.,  0.],
       [ 0.,  1.,  1., ...,  3.,  0.,  0.],
       ...,
       [ 2.,  8.,  1., ...,  2.,  1.,  0.],
       [ 0.,  1.,  9., ..., 10.,  0.,  0.],
       [ 2.,  0.,  0., ...,  0.,  0.,  0.]])

# Nearest Neighbor/Correlation

In [39]:
# Calculate the correlation
cell_norm = np.sqrt(np.sum(np.power(cell_table, 2), axis=1))
cell_corr = cell_table.dot(codebook.T) / np.reshape(cell_norm + 1e-6, (-1,1))  # add 1e-6 to avoid the denominator being 0

In [40]:
cell_corr[200:205,]

array([[1.38873008, 0.        , 0.10286889, 0.05143445, 0.20573779,
        0.77151671, 0.8743856 , 0.72008226, 0.20573779, 0.10286889],
       [0.64326749, 0.        , 0.05360562, 0.        , 0.91129561,
        0.16081687, 0.2144225 , 0.53605624, 0.10721125, 0.85768998],
       [0.04944682, 0.09889363, 0.09889363, 0.        , 0.9889363 ,
        0.14834045, 1.03838312, 0.        , 0.9889363 , 0.09889363],
       [1.38873004, 0.        , 0.15430334, 0.        , 0.        ,
        0.77151669, 0.61721335, 0.77151669, 0.        , 0.15430334],
       [0.        , 0.        , 0.        , 0.        , 0.        ,
        0.        , 0.        , 0.        , 0.        , 0.        ]])

In [55]:
cell_corr.shape

(660, 10)

In [41]:
# cell_norm + 1e-6

In [42]:
# (cell_table.dot(codebook.T) / np.reshape(cell_norm, (-1,1))).shape

In [43]:
# cell_norm

In [44]:
# np.reshape(cell_norm, (-1,1))

In [51]:
# Assign cells to barcodes according to the max correlation
cell_id = pd.DataFrame(np.argmax(cell_corr, axis=1), columns=['barcode'])
cell_id

,barcode
0,0
1,0
2,4
3,0
4,0
...,...
655,0
656,9
657,5
658,4


In [46]:
# cell_id.loc[200:210]

In [47]:
# sumGFPMasks = hf['Summed GFP Masks']
# len(np.unique(masks_mem))

In [57]:
heapq.nlargest(2, cell_corr[200,])[1]

0.8743856048116447

In [58]:
cell_id['max'] = np.max(cell_corr)
cell_id['second_max'] = [heapq.nlargest(2, cell_corr[i,])[1] for i in cell_id.index]
cell_id

,barcode,max,second_max
0,0,1.414213,0.000000
1,0,1.414213,0.000000
2,4,1.414213,1.206045
3,0,1.414213,0.000000
4,0,1.414213,0.000000
...,...,...,...
655,0,1.414213,0.946094
656,9,1.414213,0.763674
657,5,1.414213,1.005038
658,4,1.414213,0.813143


How to define significant difference between two correlations

<!-- list1 = [10, 20, 4, 45, 99]
 
# new_list is a set of list1
new_list = set(list1)
 
# removing the largest element from temp list
new_list.remove(max(new_list))
 
# elements in original list are not changed
# print(list1)
 
print(max(new_list)) -->

In [ ]:
# cell_id.to_csv('./result/crisprmap20210506_layer1out0123_cell_id_thresh6std.csv', index=False)